<a href="https://colab.research.google.com/github/nikitaoltyan/Mech-Trading-Systems/blob/main/Stochastic_parabolic_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Project cloning
!git clone https://github.com/nikitaoltyan/Mech-Trading-Systems.git
%cd Mech-Trading-Systems

Cloning into 'Mech-Trading-Systems'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 300 (delta 20), reused 7 (delta 3), pack-reused 262
Receiving objects: 100% (300/300), 1.97 MiB | 16.40 MiB/s, done.
Resolving deltas: 100% (149/149), done.
/content/Mech-Trading-Systems


In [5]:
import pandas as pd
import numpy as np
from PlotDraw import draw_plot

In [6]:
data = pd.read_excel("LHAG.xlsx")
data.head()

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M


In [7]:
data.astype({'<CLOSE>': 'int32', '<OPEN>': 'int32', '<HIGH>': 'int32', '<LOW>': 'int32'}).dtypes
data.astype({'<DATE>': 'datetime64[ns]'}).dtypes

<DATE>     datetime64[ns]
<CLOSE>           float64
<OPEN>            float64
<HIGH>            float64
<LOW>             float64
<VOL>              object
dtype: object

In [8]:
def stochastic(data, period = 14, m = 3):
    stoch_data = data.copy()
    stoch_data['14-HIGH'] = stoch_data['<HIGH>'].rolling(period).max()
    stoch_data['14-LOW'] = stoch_data['<LOW>'].rolling(period).min()
    stoch_data['%K'] = (stoch_data['<CLOSE>'] - stoch_data['14-LOW'])*100/(stoch_data['14-HIGH'] - stoch_data['14-LOW'])
    stoch_data['%D'] = stoch_data['%K'].rolling(m).mean()
    return stoch_data

In [9]:
stoch_data = stochastic(data)
stoch_data

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>,14-HIGH,14-LOW,%K,%D
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M,NaN,NaN,NaN,NaN
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M,NaN,NaN,NaN,NaN
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M,NaN,NaN,NaN,NaN
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M,NaN,NaN,NaN,NaN
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1258,"Jan 08, 2016",14.920,15.060,15.140,14.840,5.90M,15.14,13.205,88.630491,79.196200
1259,"Jan 07, 2016",15.010,14.800,15.150,14.615,8.07M,15.15,13.205,92.802057,87.479378
1260,"Jan 06, 2016",15.160,14.910,15.410,14.900,8.04M,15.41,13.205,88.662132,90.031560
1261,"Jan 05, 2016",14.925,14.670,14.995,14.625,6.52M,15.41,13.205,78.004535,86.489574


In [11]:
stoch_data = stochastic(data)

money = 100000
is_purchased = False
last_price = 0
current_shares = 0
commission = 0.003

for _, row in stoch_data.iterrows():
    buy_condition = (row["%K"]<20) and (row["%D"]<20)
    sell_condition = (row["%K"]>80) and (row["%D"]>80)
    if (buy_condition) and not (is_purchased):
        is_purchased = True
        open_price = row["<CLOSE>"]
        current_shares = money // open_price
        deal_commission = round(current_shares * open_price * commission, 5)
        money = money - (open_price * current_shares) - deal_commission
    if (sell_condition) and (is_purchased):
        is_purchased = False
        close_price = row["<CLOSE>"]
        deal_commission = round(current_shares * close_price * commission, 5)
        money = money + (close_price * current_shares) - deal_commission
        current_shares = 0
    last_price = row["<CLOSE>"]
print(money+ current_shares*last_price)

88119.37861999997


In [13]:
def calculate_profit(money, period, m):
    stoch_data = stochastic(data, period=period, m=m)

    money = money
    is_purchased = False
    last_price = 0
    current_shares = 0
    commission = 0.003

    for _, row in stoch_data.iterrows():
        buy_condition = (row["%K"]<20) and (row["%D"]<20)
        sell_condition = (row["%K"]>80) and (row["%D"]>80)
        if (buy_condition) and not (is_purchased):
            is_purchased = True
            open_price = row["<CLOSE>"]
            current_shares = money // open_price
            deal_commission = round(current_shares * open_price * commission, 5)
            money = money - (open_price * current_shares) - deal_commission
        if (sell_condition) and (is_purchased):
            is_purchased = False
            close_price = row["<CLOSE>"]
            deal_commission = round(current_shares * close_price * commission, 5)
            money = money + (close_price * current_shares) - deal_commission
            current_shares = 0
        last_price = row["<CLOSE>"]
    return money+current_shares*last_price

In [17]:
periods = [2, 30]
ms = [2, 30]
start_money = 100000
results = {}

for iter in range (128):
    period = np.random.randint(periods[0], periods[1])
    m = np.random.randint(ms[0], ms[1])
    profit = calculate_profit(start_money, period, m)
    print(f"For iter {iter}, profit {profit-start_money}. Period: {period}, m: {m}")
    
    results[profit-start_money] = (period, m)

print(f"Max profit: {max(results)} for $100k start money was received. \nUsed period: {results[max(results)]}")

For iter 0, profit 117341.40717000002. Period: 21, m: 20
For iter 1, profit 0.0. Period: 8, m: 22
For iter 2, profit 113514.92371999996. Period: 19, m: 24
For iter 3, profit -35677.18079999999. Period: 12, m: 26
For iter 4, profit 99920.51525999996. Period: 20, m: 16
For iter 5, profit 0.0. Period: 3, m: 23
For iter 6, profit -39024.33499999999. Period: 5, m: 7
For iter 7, profit -11714.7494. Period: 15, m: 29
For iter 8, profit -11161.193070000008. Period: 25, m: 2
For iter 9, profit 106421.70867999995. Period: 20, m: 28
For iter 10, profit 0.0. Period: 3, m: 18
For iter 11, profit 91454.33526000002. Period: 28, m: 21
For iter 12, profit 78952.0734. Period: 28, m: 22
For iter 13, profit 0.0. Period: 5, m: 24
For iter 14, profit 44620.327569999936. Period: 22, m: 12
For iter 15, profit 111919.55606000003. Period: 9, m: 16
For iter 16, profit -21522.93391000002. Period: 13, m: 2
For iter 17, profit -15538.645700000081. Period: 15, m: 8
For iter 18, profit 49101.60932999995. Period: 25, 

In [18]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib

Selecting previously unselected package libta-lib0.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 6.0MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp37-cp37m-linux_x86_64.whl size=1437967 sha256=14148616ef1eefee6680f79f1ebeba474c7bb1655a833d5d6d8661ebb1a12d97
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [19]:
dataSAR = data.copy()
dataSAR['SAR'] = talib.SAR(dataSAR['<HIGH>'], dataSAR['<LOW>'], acceleration=0.02, maximum=0.2)
dataSAR

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>,SAR
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M,NaN
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M,11.290000
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M,11.290000
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M,11.290000
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M,11.182360
...,...,...,...,...,...,...,...
1258,"Jan 08, 2016",14.920,15.060,15.140,14.840,5.90M,13.402016
1259,"Jan 07, 2016",15.010,14.800,15.150,14.615,8.07M,13.541055
1260,"Jan 06, 2016",15.160,14.910,15.410,14.900,8.04M,13.701949
1261,"Jan 05, 2016",14.925,14.670,14.995,14.625,6.52M,13.906915


In [23]:
dataSAR = data.copy()
dataSAR['SAR'] = talib.SAR(dataSAR['<HIGH>'], dataSAR['<LOW>'], acceleration=0.02, maximum=0.2)

money = 100000
is_purchased = False
last_price = 0
current_shares = 0
prev_row_sar = 0
commission = 0.003

for _, row in dataSAR.iterrows():
    buy_condition = (last_price > prev_row_sar) and (row["SAR"]>row["<CLOSE>"])
    sell_condition = (last_price < prev_row_sar) and (row["SAR"]<row["<CLOSE>"])
    if (buy_condition) and not (is_purchased):
        is_purchased = True
        open_price = row["<CLOSE>"]
        current_shares = money // open_price
        deal_commission = round(current_shares * open_price * commission, 5)
        money = money - (open_price * current_shares) - deal_commission
    if (sell_condition) and (is_purchased):
        is_purchased = False
        close_price = row["<CLOSE>"]
        deal_commission = round(current_shares * close_price * commission, 5)
        money = money + (close_price * current_shares) - deal_commission
        current_shares = 0
    prev_row_sar = row["SAR"]
    last_price = row["<CLOSE>"]
print(money+ current_shares*last_price)

8896.138250000025


In [25]:
def calculate_profit(money, acceleration, maximum):
    dataSAR = data.copy()
    dataSAR['SAR'] = talib.SAR(dataSAR['<HIGH>'], dataSAR['<LOW>'], acceleration=acceleration, maximum=maximum)

    money = 100000
    is_purchased = False
    last_price = 0
    current_shares = 0
    prev_row_sar = 0
    commission = 0.003

    for _, row in dataSAR.iterrows():
        buy_condition = (last_price > prev_row_sar) and (row["SAR"]>row["<CLOSE>"])
        sell_condition = (last_price < prev_row_sar) and (row["SAR"]<row["<CLOSE>"])
        if (buy_condition) and not (is_purchased):
            is_purchased = True
            open_price = row["<CLOSE>"]
            current_shares = money // open_price
            deal_commission = round(current_shares * open_price * commission, 5)
            money = money - (open_price * current_shares) - deal_commission
        if (sell_condition) and (is_purchased):
            is_purchased = False
            close_price = row["<CLOSE>"]
            deal_commission = round(current_shares * close_price * commission, 5)
            money = money + (close_price * current_shares) - deal_commission
            current_shares = 0
        prev_row_sar = row["SAR"]
        last_price = row["<CLOSE>"]
    return money+current_shares*last_price

In [27]:
axelerations = [0, 0.5]
maximums = [0, 1]
start_money = 100000
results = {}

for iter in range (128):
    axeleration = np.random.uniform(axelerations[0], axelerations[1])
    m = np.random.uniform(maximums[0], maximums[1])
    profit = calculate_profit(start_money, period, m)
    print(f"For iter {iter}, profit {profit-start_money}. Axeleration: {axeleration}, m: {m}")
    
    results[profit-start_money] = (axeleration, m)

print(f"Max profit: {max(results)} for $100k start money was received. \nUsed period: {results[max(results)]}")

For iter 0, profit -96617.6096700001. Axeleration: 0.0487091134701394, m: 0.2851780837533362
For iter 1, profit -97906.00851000007. Axeleration: 0.10020816938249172, m: 0.4935461682526989
For iter 2, profit -97817.04961000005. Axeleration: 0.11528632777507197, m: 0.38458145916655584
For iter 3, profit -92081.1044199999. Axeleration: 0.423913347086251, m: 0.10658563851192548
For iter 4, profit -97973.42555000004. Axeleration: 0.32511838766798, m: 0.5528770651357524
For iter 5, profit -98144.35353000005. Axeleration: 0.10607723438325545, m: 0.8692572513229883
For iter 6, profit -97973.42555000004. Axeleration: 0.45886675177272157, m: 0.5488322952715469
For iter 7, profit -98144.35353000005. Axeleration: 0.4420100031423845, m: 0.7198936637328187
For iter 8, profit -98144.35353000005. Axeleration: 0.05656318045488418, m: 0.7082672247694407
For iter 9, profit -92163.21900999999. Axeleration: 0.4282862073908569, m: 0.10076980490647647
For iter 10, profit -96648.00851999992. Axeleration: 0.07